In [1]:
from app_data_loader import *
from __future__ import division
from __future__ import print_function
import pandas as pd
import bokeh
import numpy as np

import bokeh.plotting as bp
from bokeh.plotting import figure, output_file, show,gridplot
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.axes import DatetimeAxis

from datetime import datetime
from datetime import timedelta

import pandas as pd

bp.output_notebook()

Loading BokehJS ...

In [2]:
#data = DataLoader('bat_data/E6_AA_91_B9_BF_48.scn.3') #

appData = AppDataLoader('app_data/879EO_1465008434236.txt') #
samplesDf = appData.jasonChunksToSamples()

("Error: number of samples doesn't match actaul array. Chunk# ", 10709)
("Error: number of samples doesn't match actaul array. Chunk# ", 10719)
("Error: number of samples doesn't match actaul array. Chunk# ", 10729)
("Error: number of samples doesn't match actaul array. Chunk# ", 10743)
("Error: number of samples doesn't match actaul array. Chunk# ", 10757)
("Error: number of samples doesn't match actaul array. Chunk# ", 10771)
("Error: number of samples doesn't match actaul array. Chunk# ", 10776)
("Error: number of samples doesn't match actaul array. Chunk# ", 10781)
("Error: number of samples doesn't match actaul array. Chunk# ", 10787)
("Error: number of samples doesn't match actaul array. Chunk# ", 10797)
("Error: number of samples doesn't match actaul array. Chunk# ", 10800)
("Error: number of samples doesn't match actaul array. Chunk# ", 10804)
("Error: number of samples doesn't match actaul array. Chunk# ", 10808)
("Error: number of samples doesn't match actaul array. Chunk# ",

In [3]:
samplesDf.head()

,badge,member,signal,voltage
datetime,,,,
2016-06-04 02:47:15.153,C8:62:29:B6:AC:B4,W6NMGEVHRE,5,2.681525
2016-06-04 02:47:15.203,C8:62:29:B6:AC:B4,W6NMGEVHRE,6,2.681525
2016-06-04 02:47:15.253,C8:62:29:B6:AC:B4,W6NMGEVHRE,5,2.681525
2016-06-04 02:47:15.303,C8:62:29:B6:AC:B4,W6NMGEVHRE,6,2.681525
2016-06-04 02:47:15.353,C8:62:29:B6:AC:B4,W6NMGEVHRE,5,2.681525


In [4]:
samplesDf.groupby(['badge']).agg(['mean', 'count','min','max'])

signal                    voltage                    \
                       mean   count min  max      mean   count       min   
badge                                                                      
C8:62:29:B6:AC:B4  6.757685  611729   1  135  2.348669  611729  1.829912   
D6:B8:C6:BA:9B:2B  4.394153  663144   0   82  2.470331  663144  2.044575   

                             
                        max  
badge                        
C8:62:29:B6:AC:B4  2.681525  
D6:B8:C6:BA:9B:2B  2.751906

# check gaps between chunks

In [5]:
timestampsMs = []
for j in range(len(appData.jasonChunks)):
    batch = {}
    batch.update(appData.jasonChunks[j])  # Create a deep copy of the jth batch of samples
    samples = batch.pop('samples')
    badge = batch.pop('badge')
    voltage = batch.pop('voltage')
    numSamples = batch.pop('numSamples')
    reference_timestamp = batch.pop('timestamp') * 1000 + batch.pop(
                'timestamp_ms')  # reference timestamp in milliseconds
    timestampsMs.append({'ts':reference_timestamp, 'badge':badge, 'voltage':voltage, 'numSamples':numSamples})
    
dfTimestampsMs = pd.DataFrame(timestampsMs)
print(dfTimestampsMs.head())

               badge  numSamples             ts   voltage
0  C8:62:29:B6:AC:B4         114  1465008435153  2.681525
1  D6:B8:C6:BA:9B:2B         114  1465008435886  2.751906
2  C8:62:29:B6:AC:B4         114  1465008440853  2.681525
3  D6:B8:C6:BA:9B:2B         114  1465008441586  2.751906
4  C8:62:29:B6:AC:B4         114  1465008446525  2.674487


In [6]:
dfTimestampsMsBadge1 = dfTimestampsMs[dfTimestampsMs.badge=='D6:B8:C6:BA:9B:2B'].copy().reset_index()
print(dfTimestampsMsBadge1.head())

   index              badge  numSamples             ts   voltage
0      1  D6:B8:C6:BA:9B:2B         114  1465008435886  2.751906
1      3  D6:B8:C6:BA:9B:2B         114  1465008441586  2.751906
2      5  D6:B8:C6:BA:9B:2B         114  1465008447255  2.751906
3      7  D6:B8:C6:BA:9B:2B         114  1465008452984  2.751906
4      9  D6:B8:C6:BA:9B:2B         114  1465008458686  2.751906


In [7]:
dfTimestampsMsBadge1.dtypes

index           int64
badge          object
numSamples      int64
ts              int64
voltage       float64
dtype: object

In [8]:
dfTimestampsMsBadge1['ts_prev'] = dfTimestampsMsBadge1['ts'].shift(-1).fillna(0.0).astype(int)
dfTimestampsMsBadge1['diff_ts'] = dfTimestampsMsBadge1['ts_prev'] - dfTimestampsMsBadge1['ts'] #dfTimestampsMsBadge1['ts'].diff()
dfTimestampsMsBadge1.head()

,index,badge,numSamples,ts,voltage,ts_prev,diff_ts
0,1,D6:B8:C6:BA:9B:2B,114,1465008435886,2.751906,1465008441586,5700
1,3,D6:B8:C6:BA:9B:2B,114,1465008441586,2.751906,1465008447255,5669
2,5,D6:B8:C6:BA:9B:2B,114,1465008447255,2.751906,1465008452984,5729
3,7,D6:B8:C6:BA:9B:2B,114,1465008452984,2.751906,1465008458686,5702
4,9,D6:B8:C6:BA:9B:2B,114,1465008458686,2.751906,1465008464396,5710


In [9]:
# Basically, you should only see 
expectedDiffMs = 50*114 + 100
dfTimestampsMsBadge1['expected']=expectedDiffMs
print(expectedDiffMs)
dfTimestampsMsBadge1[dfTimestampsMsBadge1.diff_ts > dfTimestampsMsBadge1.expected]

5800


,index,badge,numSamples,ts,voltage,ts_prev,diff_ts,expected
1218,2437,D6:B8:C6:BA:9B:2B,114,1465015378836,2.632258,1465015384649,5813,5800
2377,4755,D6:B8:C6:BA:9B:2B,114,1465021984723,2.435191,1465021991388,6665,5800
3961,7923,D6:B8:C6:BA:9B:2B,114,1465031014752,2.575953,1465031020779,6027,5800
4323,8647,D6:B8:C6:BA:9B:2B,114,1465033077748,2.157185,1465033084323,6575,5800
4339,8679,D6:B8:C6:BA:9B:2B,114,1465033169723,2.575953,1465033175524,5801,5800
5001,10003,D6:B8:C6:BA:9B:2B,114,1465036943359,2.558358,1465036949167,5808,5800
5130,10261,D6:B8:C6:BA:9B:2B,114,1465037678568,2.568915,1465037684472,5904,5800
5403,10770,D6:B8:C6:BA:9B:2B,22,1465039235001,2.340176,1465039245359,10358,5800
5548,10971,D6:B8:C6:BA:9B:2B,114,1465040066201,2.118475,1465040080185,13984,5800
5581,11030,D6:B8:C6:BA:9B:2B,98,1465040262612,2.537243,1465040281087,18475,5800
